In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import layers

Using TensorFlow backend.


In [2]:
PATH = './animefacedataset'
file_list =[]
img_list = []
#load all data
def get_data(path=PATH):
    sub_dir = os.listdir(path)
    for i in sub_dir:
        curr = path+'/'+i
        if os.path.isdir(curr):
            get_data(curr)
        else:
            file_list.append(curr)
get_data()
#check for invalid data
for i in file_list:
    try:
        tmp_img = cv2.imread(i)
        tmp_img = cv2.resize(tmp_img,(32,32))
        img_list.append(tmp_img)
    except:
        if k == 0:
            print("list of uncompatible file(s)........")
        print(i)
        k+=1
d_in = np.array(img_list)
d_in = (d_in-127.5)/127.5
print(len(d_in))

63565


In [3]:
split = train_test_split(d_in, d_in, test_size=0.3, random_state=30)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
noise_shape =(100,)
img_shape = d_in[0].shape
img_shape

In [4]:
#define discriminator model (modify cnn-pokemon)
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, 3, input_shape=(img_shape), activation='relu'))
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(16,3,strides=(2,2),activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 16)          4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               18560     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8

In [7]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(32*32*256, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((16, 16, 256)))
    assert model.output_shape == (None, 16, 16, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 32, 32, 3)

    return model

In [8]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

TypeError: The added layer must be an instance of class Layer. Found: <keras.layers.core.Dense object at 0x000001ADBF8BD9C8>